<a href="https://colab.research.google.com/github/Eliokay/Visualizing-US-Health-Insurance-Coverage/blob/main/Eloho's_Grouped_Healthcare_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import geopandas as gpd
from vega_datasets import data
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [5]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQ4mHs8RkA8Kq9_GtZizGwfyS5zWuU4g9iFVBSmzaZBLNId_6ss2lIRr-tQz3a3Mlqngx9WyUgCFsIh/pub?gid=1942231317&single=true&output=csv')
df.shape

(80818, 59)

In [7]:
df.dropna(subset = 'HCOVANY',inplace = True)
df.shape

(80818, 59)

In [10]:
df.head(5)

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,COUNTYFIP,CITY,STRATA,...,CLASSWKRD,OCC,INCTOT,FTOTINC,POVERTY,VETDISAB,DIFFREM,DIFFPHYS,VETSTAT,VETSTATD
0,2022,202201,1429400,2022010000059,17,2022014294001,53,33,0,2330153,...,22,9620,8200,9999999,0,0,1,1,1,11
1,2022,202201,1429401,2022010000078,1,2022014294011,53,0,0,2270053,...,27,4220,4600,9999999,0,0,1,1,1,11
2,2022,202201,1429402,2022010000113,21,2022014294021,53,33,6430,2331753,...,22,4720,6700,9999999,0,0,1,1,1,11
3,2022,202201,1429403,2022010000120,18,2022014294031,53,33,0,2330753,...,0,0,10100,9999999,0,0,2,2,1,11
4,2022,202201,1429404,2022010000159,13,2022014294041,53,63,0,2630353,...,0,0,890,9999999,0,0,1,2,1,11


In [11]:
adults = df[df['AGE']>=18]
adults2022 = adults[adults['YEAR'] == 2022]
adults2022.shape

(64960, 59)

In [12]:
adults['COUNTYFIP'].value_counts()

33    17952
0     14381
53     7216
61     6522
63     4525
11     3824
35     2466
67     2235
73     2017
57     1942
77     1880
Name: COUNTYFIP, dtype: int64

In [13]:
bycategory = adults.groupby(['COUNTYFIP','CITY',"SEX", "AGE", "MARRNO", "RACE", "RACED", "BPL", "BPLD", "CITIZEN", "LANGUAGE", "LANGUAGED", "SPEAKENG", "RACAMIND", "RACASIAN", "RACBLK", "RACPACIS", "RACWHT", "RACOTHER", "HCOVANY", "HCOVPRIV", "HINSEMP", "HINSPUR", "HINSTRI", "HCOVPUB", "HINSCAID", "HINSCARE", "HINSVA", "HINSIHS", "SCHOOL", "EDUC", "EDUCD", "SCHLTYPE", "DEGFIELD", "DEGFIELDD", "EMPSTAT", "EMPSTATD", "CLASSWKR", "CLASSWKRD", "OCC", "INCTOT", "FTOTINC", "POVERTY", "VETDISAB", "DIFFREM", "DIFFPHYS", "VETSTAT", "VETSTATD"],as_index=False).count()
bycategory.head()

,COUNTYFIP,CITY,SEX,AGE,MARRNO,RACE,RACED,BPL,BPLD,CITIZEN,...,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,STRATA,GQ,PERNUM,PERWT
0,0,0,1,18,0,1,100,4,400,0,...,1,1,1,1,1,1,1,1,1,1
1,0,0,1,18,0,1,100,5,500,0,...,1,1,1,1,1,1,1,1,1,1
2,0,0,1,18,0,1,100,6,600,0,...,1,1,1,1,1,1,1,1,1,1
3,0,0,1,18,0,1,100,6,600,0,...,1,1,1,1,1,1,1,1,1,1
4,0,0,1,18,0,1,100,6,600,0,...,1,1,1,1,1,1,1,1,1,1


Plot 1. Health Insurance Coverage by Duration of Marriage and Race in Washington-HeatMap

In [14]:
# Group data by marriage status, change how marriage variables are named to make it understandable
bymarriage = adults.groupby(['MARRNO', 'RACE'],as_index=False)['HCOVANY'].mean()
bymarriage['HCOVANY'] = bymarriage['HCOVANY'] -1

bymarriage.loc[bymarriage['RACE'] == 1, 'RACE'] = 'White'
bymarriage.loc[bymarriage['RACE'] == 2, 'RACE'] = 'Black'
bymarriage.loc[bymarriage['RACE'] == 3, 'RACE'] = 'Native'
bymarriage.loc[bymarriage['RACE'] == 4, 'RACE'] = 'Chinese'
bymarriage.loc[bymarriage['RACE'] == 5, 'RACE'] = 'Japanese'
bymarriage.loc[bymarriage['RACE'] == 6, 'RACE'] = 'Pacific Islander'
bymarriage.loc[bymarriage['RACE'] == 7, 'RACE'] = 'Other Race'
bymarriage.loc[bymarriage['RACE'] == 8, 'RACE'] = 'Two major races'
bymarriage.loc[bymarriage['RACE'] == 9, 'RACE'] = 'Three or more major races'
# bymarriage.head()

# Plot map, add text on map to aid the understanding of the color encoding which is telling us percentage of health insurance coverage
base=alt.Chart(bymarriage).mark_rect(
).encode(
    alt.Y('RACE:N', title=''),
    alt.X('MARRNO:O', title='Number of years Married'),
    alt.Color('HCOVANY:Q', legend = None, )
).properties(width = 800, height = 800, title="Health Insurance Coverage by Duration of Marriage and Race in Washington")

text = base.mark_text(baseline='middle', size=20).encode(
    alt.Text('HCOVANY:Q', format=".0%"),
    color=alt.condition(
        alt.datum.HCOVANY < .80,
        alt.value('black'),
        alt.value('white')
    )
)

Finalheatmap = (base + text)
Finalheatmap.configure_title(
     fontSize=25,
     anchor='middle',
).configure_header(
    labelFontSize=20
).configure_axis(
    titleFontSize=18,
    labelFontSize=18,
    gridOpacity=0.2

)

alt.LayerChart(...)


Plot 2. Health Insurance Coverage By Counties in Washington- Choropleth Map

In [15]:
#Group data by county fips and aggregate insurance coverage to a percentage
byFIPS = adults.groupby(['COUNTYFIP'],as_index=False)['HCOVANY'].mean()
byFIPS['HCOVANY'] = byFIPS['HCOVANY'] -1
# byFIPS.tail()

#Use geopandas to get base map of washington state and match the county fips code in the original dataset to the toptjson file. Use string match to match the last hree digits as the fips codes are not coded the same
countydf = gpd.read_file(data.us_10m.url,driver='TopoJSON')
countydf.head()
countydf['WAFIPS'] = countydf['id'].str.extract('(\d{2})$')
washington = countydf[countydf['id'].str.match('53\d*')]

byFIPS.dropna(subset = 'HCOVANY',inplace = True)
byFIPS['COUNTYFIP'] = byFIPS['COUNTYFIP'].astype(int)
washington['WAFIPS'] = washington['WAFIPS'].astype(int)

mapdata = byFIPS.merge(washington, left_on = 'COUNTYFIP', right_on='WAFIPS', how='outer')
# mapdata.head(12)


#Plot map and resize to make it effective
basemap=alt.Chart(washington).mark_geoshape().encode(color=alt.value('lightgray'))

bm2=basemap.encode(
    color = alt.condition("datum.HCOVANY !== 'null'", 'HCOVANY:Q', alt.value('lightgray')),
    tooltip=['WAFIPS:N', 'HCOVANY:Q']
).transform_lookup(
    lookup='WAFIPS',
    from_=alt.LookupData(byFIPS, 'COUNTYFIP', ["HCOVANY"]),
).project(
    type='albersUsa'
).properties(
    width=800,
    height=600,
    title='Health Insurance Coverage By Counties in Washington'
)


FinalMap = ((basemap + bm2).resolve_scale(color='independent'))
FinalMap
FinalMap.configure_title(
     fontSize=30,
     anchor='middle').configure_header(
    labelFontSize=20
).configure_axis(
    titleFontSize=16,
    labelFontSize=10)

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


alt.LayerChart(...)

Plot 3. Education Level in Relation to Health Insurance Coverage - Bar chart

In [16]:
#Make nnew group by education level
byeducation = adults.groupby(['EDUC'],as_index=False)['HCOVANY'].mean()
byeducation['HCOVANY'] = byeducation['HCOVANY'] -1

#Change how the variables are named so it would be easy to understand
byeducation.loc[byeducation['EDUC'] == 0o0, 'EDUC'] = 'No Schooling'
byeducation.loc[byeducation['EDUC'] == 0o1, 'EDUC'] = 'Nursery school to grade 4'
byeducation.loc[byeducation['EDUC'] == 0o2, 'EDUC'] = 'Grade 5, 6, 7, or 8'
byeducation.loc[byeducation['EDUC'] == 0o3, 'EDUC'] = 'Grade 9'
byeducation.loc[byeducation['EDUC'] == 0o4, 'EDUC'] = 'Grade 10'
byeducation.loc[byeducation['EDUC'] == 0o5, 'EDUC'] = 'Grade 11'
byeducation.loc[byeducation['EDUC'] == 0o6, 'EDUC'] = 'Grade 12'
byeducation.loc[byeducation['EDUC'] == 0o7, 'EDUC'] = '1 year of college'
# byeducation.loc[byeducation['EDUC'] == 0o8, 'EDUC'] = '2 year of college'
# byeducation.loc[byeducation['EDUC'] == 0o9, 'EDUC'] = '3 year of college'
byeducation.loc[byeducation['EDUC'] == 10, 'EDUC'] = '4 years of college'
byeducation.loc[byeducation['EDUC'] == 11, 'EDUC'] = '5+ years of college'
byeducation.loc[byeducation['EDUC'] == 99, 'EDUC'] = 'Missing'
# byeducation.head()

# Plot map
base = alt.Chart(byeducation).mark_bar().encode(
    alt.Y('EDUC:N', scale=alt.Scale(domain=['No Schooling','Grade 5, 6, 7, or 8', 'Grade 9', '1 year of college', '4 years of college', '5+ years of college']), title=""),
    alt.X('HCOVANY:Q', axis=alt.Axis(format='%'), title="Health Insurance Coverage"),
    text = "HCOVANY",
).properties(width=500,height=500, title='Education Level in Relation to Health Insurance Coverage')
base.configure_title(
     fontSize=25,
     anchor='middle',
).configure_header(
    labelFontSize=20
).configure_axis(
    titleFontSize=16,
    labelFontSize=18,
    gridOpacity=0
)

alt.Chart(...)